# Decision Tree implementation

The follwing decision tree code can be used to predict the final grades of a person whose course wise grades are given

In [56]:
import csv
import numpy
import numpy
import sys
import warnings
warnings.filterwarnings("error")

In [57]:
trainin='education_train.csv'
testin = 'education_test.csv'
maxdepth = 5
trainout = 'train_out.txt'
testout = 'test_out.txt'
metricsout = 'metrics_out.txt'

In [58]:
f = open(trainin)
reader = csv.reader(f,delimiter=',')
i=0
for row in reader:
    print(row)
    i=i+1
    if i>=10:
        break
f.close()

['M1', 'M2', 'M3', 'M4', 'M5', 'P1', 'P2', 'P3', 'P4', 'F', 'grade']
['notA', 'notA', 'A', 'notA', 'A', 'A', 'A', 'notA', 'notA', 'A', 'A']
['notA', 'A', 'A', 'notA', 'A', 'notA', 'notA', 'notA', 'A', 'notA', 'notA']
['notA', 'A', 'A', 'A', 'A', 'notA', 'A', 'notA', 'notA', 'A', 'A']
['notA', 'notA', 'notA', 'notA', 'A', 'A', 'A', 'notA', 'notA', 'A', 'notA']
['A', 'notA', 'A', 'A', 'A', 'A', 'A', 'notA', 'notA', 'A', 'A']
['notA', 'notA', 'A', 'notA', 'A', 'notA', 'A', 'notA', 'notA', 'notA', 'notA']
['notA', 'A', 'A', 'notA', 'notA', 'A', 'A', 'A', 'notA', 'notA', 'notA']
['notA', 'notA', 'A', 'notA', 'A', 'A', 'A', 'A', 'notA', 'notA', 'notA']
['notA', 'A', 'A', 'A', 'notA', 'A', 'A', 'A', 'notA', 'notA', 'A']


In the train file there are 10 features with binary values (A or notA). The final column corresponds t the final grade

The following is the decision tree code for the prediction

In [59]:
def decisionTree(trainin,testin,maxdepth,trainout,testout,metricsout):

    def readfile():
        M=[]
        N=[]
        # with open('education_train.csv') as f:
        # with open('small_train.csv') as f:
        with open(trainin) as f:
            reader = csv.reader(f,delimiter=',')
            i=0
            for row in reader:
                M.append(row)
                i=i+1

        with open(testin) as g:
            reader = csv.reader(g,delimiter=',')
            i=0
            for row in reader:
                N.append(row)
                i=i+1
        return M,N

    def values(M):
        out = ['output1', 'output2']
        out[0] = M[1][-1]
        attn = ['attn1', 'attn2']
        attn[0] = M[1][1]
        for j in range(2,len(M)):
            if (M[j][-1] != out[0]):
                out[1]=M[j][-1]
            if out[1]!='output2':
                break
        for j in range(2,len(M)):
            if (M[j][1] != attn[0]):
                attn[1]=M[j][1]
            if attn[1]!='attn2':
                break
        return [out,attn]
    def rootnodef(M,out,attn):
        IYall = []
        nl=[]
        used=[]
        for j in range(0,len(M[1])-1):
            v00 = 0.0 
            v01 = 0.0 
            v10 = 0.0 
            v11 = 0.0 

            for i in range(1,len(M)):
                if M[i][j] == attn[0]:
                    if M[i][-1] == out[0]:
                        v00 +=1
                    else:
                        v01 +=1
                else:
                    if M[i][-1] == out[0]:
                        v10 +=1
                    else:
                        v11 +=1
            ts0=v00+v10
            ts1=v01+v11
            t0s=v00+v01
            t1s=v10+v11

            try:
                HY=(-ts0/(ts0+ts1))*(numpy.log2(ts0/(ts0+ts1)))+(-ts1/(ts0+ts1))*(numpy.log2(ts1/(ts0+ts1)))
            except:
                HY=0.0
            try:
                HYn=(-v00/(v00+v01))*(numpy.log2(v00/(v00+v01)))+(-v01/(v00+v01))*(numpy.log2(v01/(v00+v01)))
            except:
                HYn=0.0
            try:
                HYy=(-v10/(v10+v11))*(numpy.log2(v10/(v10+v11)))+(-v11/(v10+v11))*(numpy.log2(v11/(v10+v11)))
            except:
                HYy=0.0
            IY=HY-(t0s/(ts0+ts1))*HYn-(t1s/(ts0+ts1))*HYy

            IYall.append(IY)

            if (v00 > v01):
                decl = -2 #attn[0]
            else:
                decl = -1 #attn[1]

            if (v10 > v11):
                decr = -2 #attn[0]
            else:
                decr = -1 #attn[1]

            val0 = []
            val1 = []
            for i in range(1, len(M)):
                if M[i][j] == attn[0]:
                    val0.append(M[i])
                else:
                    val1.append(M[i])
            data = [val0, val1]
            ati = j
            used=[j]
            nl.append([0, ati, [v00, v01], [v10, v11], IY, [decl, decr], data,used])
        finalnode = nl[IYall.index(max(IYall))]

        return finalnode

    M,N=readfile()
    X,Y=readfile()

    out=values(M)[0]
    attn=values(M)[1]

    def allatcal():
        attarr = []
        for j in range(0, len(M[1]) - 1):
            attnew = ['attn1', 'attn2']
            attnew[0] = M[1][j]

            for i in range(2, len(M)):
                if (M[i][j] != attnew[0]):
                    attnew[1] = M[i][j]
                if attnew[1] != 'attn2':
                    break
            attarr.append(attnew)
        return attarr

    attarr = allatcal()

    def attchng(attarr, attn, X):
        for j in range(0, len(X[1]) - 1):
            for i in range(1, len(X)):
                if X[i][j] == attarr[j][0]:
                    X[i][j] = attn[0]
                elif X[i][j] == attarr[j][1]:
                    X[i][j] = attn[1]
                    # print X[i][j]
        return X

    # for j in range(0,len(M[0])-1):
    X = attchng(attarr, attn, X)
    Y = attchng(attarr, attn, Y)

    M = X
    N = Y

    hei = int(maxdepth)

    if hei>len(M[0])-2:
        hei=len(M[0])-2
    else:
        po='The height is within limits'

    rootnode=rootnodef(M,out,attn)

    def dectree(M,rootnode):
        dtree=[]
        dtree.append([rootnode[0],rootnode[1]])
        tree=[]
        tree.append(rootnode)
        def attfind(M,tree,pnode,dir):
            ldata = pnode[6][dir]
            h=pnode[0]+1
            IYall=[]
            nl=[]
            used=pnode[7]

            for j in range(0,len(M[1])-1):
                if j in used:
                    continue
                else:
                    v00 = 0.0
                    v01 = 0.0
                    v10 = 0.0
                    v11 = 0.0
                    for i in range(0, len(ldata)):
                        if ldata[i][j] == attn[0]:
                            if ldata[i][-1] == out[0]:
                                v00 += 1
                            else:
                                v01 += 1
                        else:
                            if ldata[i][-1] == out[0]:
                                v10 += 1
                            else:
                                v11 += 1
                    ts0 = v00 + v10
                    ts1 = v01 + v11
                    t0s = v00 + v01
                    t1s = v10 + v11

                    try:
                        HY = (-ts0 / (ts0 + ts1)) * (numpy.log2(ts0 / (ts0 + ts1))) + (-ts1 / (ts0 + ts1)) * (
                            numpy.log2(ts1 / (ts0 + ts1)))
                    except:
                        HY = 0.0
                    try:
                        HYn = (-v00 / (v00 + v01)) * (numpy.log2(v00 / (v00 + v01))) + (-v01 / (v00 + v01)) * (
                            numpy.log2(v01 / (v00 + v01)))
                    except:
                        HYn = 0.0
                    try:
                        HYy = (-v10 / (v10 + v11)) * (numpy.log2(v10 / (v10 + v11))) + (-v11 / (v10 + v11)) * (
                            numpy.log2(v11 / (v10 + v11)))
                    except:
                        HYy = 0.0
                    IY = HY - (t0s / (ts0 + ts1)) * HYn - (t1s / (ts0 + ts1)) * HYy
                    IYall.append(IY)

                    if (v00 > v01):
                        decl = -2 #attn[0]
                    else:
                        decl = -1 #attn[1]

                    if (v10 > v11):
                        decr = -2 #attn[0]
                    else:
                        decr = -1 #attn[1]

                    val0 = []
                    val1 = []

                    for i in range(0, len(ldata)):
                        if ldata[i][j] == attn[0]:
                            val0.append(ldata[i])
                        else:
                            val1.append(ldata[i])
                    data = [val0, val1]
                    ati = j
                    nl.append([h, ati, [v00, v01], [v10, v11], IY, [decl, decr], data,used])
            finalnode=nl[IYall.index(max(IYall))]
            finalnode[7]=finalnode[7]+[finalnode[1]]
            return finalnode

        asd=0
        def tp():
            j=0
            for n in tree:
                try:
                    print([n[0],n[1],n[2],n[3],n[7],n[5]])
                except:
                    try:
                        print([n[0],n[1]])
                    except:
                        print(n)
            return

        i=0
        for node in tree:
            if node[0]>=len(M[0])-2:
                break

            if node[0]<=len(M[0])-1:
                if node[1] is -1:
                    tree.append([node[0]+1,node[1]])
                    tree.append([node[0]+1,node[1]])
                    i=i+1
                    continue
                elif node[1] is -2:
                    tree.append([node[0]+1,node[1]])
                    tree.append([node[0]+1,node[1]])
                    i=i+1
                    continue
                else:
                    direc=[0,1]
                    for dir in direc:
                        index=[2,3]
                        ind=index[dir]
                        if (0 in node[ind]):
                            tree.append([node[0]+1,node[5][dir]])
                            continue
                        elif (0 not in node[ind]):
                            try:
                                chn = attfind(M, tree, tree[i], dir)
                                if chn[4]<=0:
                                    tree.append(tree[i][0]+1,tree[i][5][dir])
                                else:
                                    tree.append(chn)
                                    continue
                            except:
                                tree.append([chn[0],chn[5][dir]])
            else:
                print('asdsa')
                continue
            i=i+1

        return tree
    tree=dectree(M,rootnode)

    def tp1():
        j = 0
        for n in tree:
            try:
                print([n[0], n[1], n[2], n[3], n[4], n[5]])
            except:
                try:
                    print([n[0], n[1]])
                except:
                    print(n)
        return
    def trim(tree,hei):
        treenew=[]
        for sh in range(0,2**hei-1):
            treenew.append(tree[sh])
        return treenew

    tree = trim(tree, hei + 1)

    def testtest(N,tree,hei):
        if hei>len(N[0])-2:
            res=1
        else:
            res=[]
            for i in range(1,len(N)):
                xr=[]
                for j in range(0,len(N[0])-1):
                    xr.append(N[i][j])
                yr=N[i][-1]
                dec=False

                n=0
                while dec==False:
                    if tree[n][1]==-1:
                        dec=True
                        res.append(out[-1])
                    elif tree[n][1]==-2:
                        dec=True
                        res.append(out[-2])
                    elif hei==0:
                        dec=True
                        atc = tree[n][1]
                        if xr[atc]==attn[0]:
                            res.append(out[tree[0][5][0]])
                            break
                        else:
                            res.append(out[tree[0][5][1]])
                            break
                    elif tree[n][0]>=hei:
                        dec=True
                        a=m[0]
                        b=m[1]
                        res.append(out[tree[a][5][b]])
                    else:
                        atc=tree[n][1]
                        if xr[atc]==attn[0]:
                            m=[n,0]
                            n=(2*n)+1
                        else:
                            m=[n,1]
                            n=(2*n)+2
        return res

    def errcal(N,fd):
        cp=0.0
        ip=0.0
        for i in range(1,len(N)):
            if N[i][-1]==fd[i-1]:
                cp+=1
            else:
                ip+=1
        err=ip/(ip+cp)
        return err

    testoutputdata=testtest(N,tree,hei)
    trainoutputdata=testtest(M,tree,hei)
    errtest=errcal(N,testoutputdata)
    errtrain=errcal(M,trainoutputdata)

    f = open(trainout, 'w')
    for d in (trainoutputdata):
        f.write(d+'\n')
    f.close()

    f = open(testout, 'w')
    for d in (testoutputdata):
        f.write(d+'\n')
    f.close()

    f = open(metricsout, 'w')
    f.write('error(train): ' + repr(errtrain)+ '\n' + 'error(test): ' + repr(errtest))
    f.close()
#     return [repr(errtrain),repr(errtest)]


Exception ignored in: <_io.FileIO name='education_test.csv' mode='rb' closefd=True>
Exception ignored in: <_io.FileIO name='train_out.txt' mode='wb' closefd=True>
Exception ignored in: <_io.FileIO name='test_out.txt' mode='wb' closefd=True>
Exception ignored in: <_io.FileIO name='metrics_out.txt' mode='wb' closefd=True>
Exception ignored in: <_io.FileIO name='train_out.txt' mode='wb' closefd=True>
Exception ignored in: <_io.FileIO name='test_out.txt' mode='wb' closefd=True>
Exception ignored in: <_io.FileIO name='metrics_out.txt' mode='wb' closefd=True>
Exception ignored in: <_io.FileIO name='education_train.csv' mode='rb' closefd=True>
Exception ignored in: <_io.FileIO name='education_test.csv' mode='rb' closefd=True>
Exception ignored in: <_io.FileIO name='train_out.txt' mode='wb' closefd=True>
Exception ignored in: <_io.FileIO name='test_out.txt' mode='wb' closefd=True>
Exception ignored in: <_io.FileIO name='metrics_out.txt' mode='wb' closefd=True>


In [60]:
decisionTree(trainin,testin,maxdepth,trainout,testout,metricsout)

# Metric

In [61]:
metrics=open(metricsout)
for line in metrics:
    print(line)

error(train): 0.085

error(test): 0.09


Exception ignored in: <_io.FileIO name='metrics_out.txt' mode='rb' closefd=True>
